# Assignment 4: Conjoint Analysis
The objective of this assignment is to give you experience with ratings-based conjoint analysis.

### Part 1: Study Design

In [6]:
# Import the experimental design builder script.
import sys
sys.path.insert(1, '/data/Assignment-4')
import designer

**Task 1:** Define a variable called `attributes` that contains the attributes and levels within each attribute.  The viable shoukd be a dictionary where each attribute is a key and the value associated witht he key is a list of the levels.  For example, if the two attributes were color (with levels red and green) and shape (with levels round and square), then `attributes` would be defined as follows.

```
attributes = {"color": ["red", "green"],
             "shape": ["round", "square"]}
```

In [7]:
attributes = {"Transparency": ["Transparent","Non-transparent"],
              "Volume": ["20 FL OZ","12 FL OZ"],
              "Sugar": ["With Added Sugar","Without Added Sugar"],
              "Texture": ["Juicy","Thick"],
              "Flavor": ["Single-fruit Flavor","Mixed-fruit Flavor"]}

In [8]:
# Use the designer script to find a matching design.
design = designer.get_design(attributes)
design
design2 = design.assign(product = [str(i) for i in range(1,13)]) 
design2

,Transparency,Volume,Sugar,Texture,Flavor,product
0,Non-transparent,20 FL OZ,Without Added Sugar,Thick,Mixed-fruit Flavor,1
1,Non-transparent,12 FL OZ,Without Added Sugar,Thick,Mixed-fruit Flavor,2
2,Transparent,20 FL OZ,Without Added Sugar,Juicy,Mixed-fruit Flavor,3
3,Transparent,20 FL OZ,With Added Sugar,Thick,Single-fruit Flavor,4
4,Transparent,12 FL OZ,With Added Sugar,Thick,Mixed-fruit Flavor,5
5,Non-transparent,20 FL OZ,With Added Sugar,Thick,Single-fruit Flavor,6
6,Non-transparent,12 FL OZ,Without Added Sugar,Juicy,Single-fruit Flavor,7
7,Transparent,12 FL OZ,With Added Sugar,Juicy,Mixed-fruit Flavor,8
8,Non-transparent,20 FL OZ,With Added Sugar,Juicy,Mixed-fruit Flavor,9
9,Transparent,20 FL OZ,Without Added Sugar,Juicy,Single-fruit Flavor,10


### Parts 2 & 3: Write and administer the survey
**Task 2:** When you are done administering the survey:
- download CSV from qualtrics
- rename the file `survey.csv` and upload it here, and
- read the CSV file in as a pandas data frame called `raw_survey_results`.

In [9]:
import pandas as pd
raw_survey_results = pd.read_csv("survey.csv",header =1)

**Task 3:** Clean the data in preparation for analysis; name the clean data frame `clean_survey_results`.  You may need to do some or all of the following:
- remove rows and/or columns
- rename columns
- melt the data
- merge the data with the design data frame you created previously
The final data frame should have one row per rating with the following columns:
- some kind of respondent identifier to distinguish individuals
- the rating (1-7)
- one column for each attribute

In [10]:
import pandas as pd
import numpy as np
clean_survey_results = raw_survey_results.drop(raw_survey_results.index[0])
clean_survey_results = clean_survey_results.drop(clean_survey_results.columns[0:8], axis=1)
clean_survey_results = clean_survey_results.drop(clean_survey_results.columns[1:9], axis=1)
clean_survey_results = clean_survey_results.drop(clean_survey_results.columns[13:], axis=1)
clean_survey_results.columns = ['Response ID'] +([str(i) for i in range(1,13)])
clean_survey_results = clean_survey_results.drop([1,2,3,4,5])
clean_survey_results = clean_survey_results.drop([28,29,30])
clean_survey_results = pd.melt(clean_survey_results, id_vars = ['Response ID'], 
                               var_name='product', value_name='rating')
clean_survey_results = clean_survey_results.merge(design2, on='product')
clean_survey_results = clean_survey_results.drop('product', axis=1)
clean_survey_results

,Response ID,rating,Transparency,Volume,Sugar,Texture,Flavor
0,R_21cD7IBjjMfnequ,2,Non-transparent,20 FL OZ,Without Added Sugar,Thick,Mixed-fruit Flavor
1,R_2PvZT1wkMwd46OK,2,Non-transparent,20 FL OZ,Without Added Sugar,Thick,Mixed-fruit Flavor
2,R_2VpwEJ8FEy5qDTG,2,Non-transparent,20 FL OZ,Without Added Sugar,Thick,Mixed-fruit Flavor
3,R_3sgnaIzKjam1RRs,3,Non-transparent,20 FL OZ,Without Added Sugar,Thick,Mixed-fruit Flavor
4,R_5oRSzEhrVqUX4Sl,6,Non-transparent,20 FL OZ,Without Added Sugar,Thick,Mixed-fruit Flavor
...,...,...,...,...,...,...,...
259,R_r0f1bjxpcCAQlUt,5,Transparent,12 FL OZ,Without Added Sugar,Thick,Single-fruit Flavor
260,R_1hTnkz1lvugm5SX,1,Transparent,12 FL OZ,Without Added Sugar,Thick,Single-fruit Flavor
261,R_1hKZwDhNyFkPdLB,4,Transparent,12 FL OZ,Without Added Sugar,Thick,Single-fruit Flavor
262,R_2X6UPTw40psPglU,4,Transparent,12 FL OZ,Without Added Sugar,Thick,Single-fruit Flavor


**Task 4:** Introduce dummy variables in preparation for anlaysis.  Now, for each attribute, there should be the number of levels minus one.  (The intercept captures the "default" level, and the part-worths for all other levels will be measured relative to the default utility.)

In [11]:
data = clean_survey_results
data['Transparency_alt'] = data['Transparency'] == "Transparent"
# e.g., if there were a third soda Brand, we would need something like this:
# data['brand_alt2'] = data['brand'] == "Dr. Pepper"
data['Volume_alt'] = data['Volume'] == "12 FL OZ"
data['Sugar_alt'] = data['Sugar'] == "Without Added Sugar"
data['Texture_alt'] = data['Texture'] == "Thick"
data['Flavor_alt'] = data['Flavor'] == "Single-fruit Flavor"
data = data.drop(['Transparency', 'Volume', 'Sugar','Texture','Flavor'], axis=1)
data.head(5)

,Response ID,rating,Transparency_alt,Volume_alt,Sugar_alt,Texture_alt,Flavor_alt
0,R_21cD7IBjjMfnequ,2,False,False,True,True,False
1,R_2PvZT1wkMwd46OK,2,False,False,True,True,False
2,R_2VpwEJ8FEy5qDTG,2,False,False,True,True,False
3,R_3sgnaIzKjam1RRs,3,False,False,True,True,False
4,R_5oRSzEhrVqUX4Sl,6,False,False,True,True,False


### Part 4: Estimate the average and individual part-worths

You will use [sklearn's implementation of linear regression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html) to estimate part-worths.

In [12]:
import numpy as np
from sklearn.linear_model import LinearRegression

**Task 5:** Compute the average part-worths by:
- splitting the data into `y` (the response/rating) and `X` (the explanatory variables/attribute levels), and
- Run a linear regression to estimate intercept and coefficients.

In [13]:
y = np.array(data.to_numpy()[:,1])
X = np.array(data.to_numpy()[:,2:], dtype=int)
reg = LinearRegression().fit(X, y)
reg.intercept_

3.492424242424242

In [14]:
reg.coef_

array([ 0.72727273,  0.13636364,  0.57575758, -1.16666667,  0.42424242])

**Task 6:** Calculate the importance of each attribute by doing the following steps.
1. Compute the part-worth range for each attribute. 
2. Calculate the total range, summing the ranges over each attribute.
3. Calculate the importance for a single attribute as the range for that attribute over the total range.

In [15]:
ranges = abs(reg.coef_)
ranges

array([0.72727273, 0.13636364, 0.57575758, 1.16666667, 0.42424242])

In [16]:
ranges / sum(ranges)

array([0.24 , 0.045, 0.19 , 0.385, 0.14 ])

**Task 7:** Now calculate individual part-worths.  You will do this by dividing up the data by individual respondent and running one regression for each respondent.  You should write a loop to iterate over all the respondents instead of doing each one manually.

You will want to create a data frame called `individual_part_worths` that has one row per individual respondent; the columns should be the respondent identifier, the intercept of the regression, and all the coefficients of the regression.

In [17]:
individual_part_worths = pd.DataFrame(columns = ["Response ID", "Intercept", "c1", "c2", "c3", "c4", "c5"])
grouped_data = data.groupby(by="Response ID")
for id, group in grouped_data:
    y = np.array(group.to_numpy()[:,1])
    X = np.array(group.to_numpy()[:,2:], dtype=int)
    reg = LinearRegression().fit(X, y)
    individual_part_worths.loc[individual_part_worths.shape[0]+1] = [id,reg.intercept_] + reg.coef_.tolist()
print(individual_part_worths)

          Response ID  Intercept            c1            c2        c3  \
1   R_1MYb8NRVqWDic9s   5.500000  1.166667e+00 -1.666667e-01 -0.833333   
2   R_1hKZwDhNyFkPdLB   5.166667  6.666667e-01  3.333333e-01 -0.666667   
3   R_1hTnkz1lvugm5SX   2.000000 -2.000000e+00  3.333333e-01  2.000000   
4   R_1jNYCTi8jqmXLua   3.833333  1.333333e+00  6.666667e-01  0.666667   
5   R_1li1A1KUljAqBXd   1.666667  1.500000e+00 -8.333333e-01  1.833333   
6   R_21alOwOj62tYfuY   3.333333  5.000000e-01  1.666667e-01 -0.166667   
7   R_21cD7IBjjMfnequ   5.166667 -3.333333e-01 -2.220446e-16 -1.666667   
8   R_21ccNHOq6XeTmQA   4.666667  0.000000e+00  6.666667e-01 -0.333333   
9   R_23VnLa7MO6QGfeB   0.833333  2.333333e+00 -3.333333e-01  1.666667   
10  R_2OMYNx5tkI5SfiQ   1.333333  2.166667e+00 -1.666667e-01  1.166667   
11  R_2PvZT1wkMwd46OK   3.833333  1.000000e+00 -3.333333e-01 -0.333333   
12  R_2VpwEJ8FEy5qDTG   1.666667  2.333333e+00 -3.333333e-01  2.000000   
13  R_2X6UPTw40psPglU   5.000000  0.00

**Task 8:** Create a visualization of the individual respondent coefficients and intercepts.  On the x axis should have categories for each of the level dummy variables and default; on the y-axis should be the part-worths (the values of the coefficients and the intercepts).  You should ploth both a summary of the part-worths (e.g., a violin plot, or box and whiskers) and the individual values.

In [ ]:
# Set up R, if desired for visualization
import rpy2.ipython
%reload_ext rpy2.ipython

In [15]:
individual_part_worths_plot = pd.melt(individual_part_worths, id_vars = ['Response ID'], value_vars = ["c1","c2","c3","c4","c5","Intercept"],var_name='Variable', value_name='Value')
individual_part_worths_plot

,Response ID,Variable,Value
0,R_1MYb8NRVqWDic9s,c1,1.166667
1,R_1hKZwDhNyFkPdLB,c1,0.666667
2,R_1hTnkz1lvugm5SX,c1,-2.000000
3,R_1jNYCTi8jqmXLua,c1,1.333333
4,R_1li1A1KUljAqBXd,c1,1.500000
...,...,...,...
127,R_5oRSzEhrVqUX4Sl,Intercept,4.000000
128,R_SB5dxiOGOn6sBjj,Intercept,3.833333
129,R_abhnQl7JF8hyMwx,Intercept,5.666667
130,R_d3WSgKkK6YMRmzT,Intercept,4.000000


In [25]:
%%R -i individual_part_worths_plot
library(ggplot2)
# Basic box plot
p_box <- ggplot(individual_part_worths_plot, aes(x=Variable, y=Value)) + geom_hline(yintercept=mean(individual_part_worths_plot$Intercept), linetype="dashed", color = "red")+geom_hline(yintercept=0, color = "red")+geom_boxplot()
p_box

UsageError: Cell magic `%%R` not found.


In [ ]:
%%R
ggsave("Summary of Part-worths.pdf", width=6.5, height=3)

In [23]:
%%R -i individual_part_worths_plot
library(ggplot2)
# Basic box plot
p_scatter <- ggplot(individual_part_worths_plot, aes(x=Variable, y=Value)) + geom_hline(yintercept=mean(individual_part_worths_plot$Intercept), linetype="dashed", color = "red")+geom_hline(yintercept=0, color = "red")+geom_point()
p_scatter

UsageError: Cell magic `%%R` not found.


In [ ]:
%%R
ggsave("Individual Value of Part-worths.pdf", width=6.5, height=3)

### Part 5: Simulate the Market
**Task 9:** Write a function `utility` that takes two arguments:
- `product`: a list of which level has been selected for each attribute.  For example, if the two attributes were color (with levels red and green) and shape (with levels round and square), then `product` could take the value `["red", "square"]`.
- `part_worths`: a row from the `individual_part_worths` data frame created for Task 6 (i.e., one individual's part-worths) .

The `utility` function should compute and return the utility of the specified individual for the specified product; this is the intercept plus any relevant part-worths that correspond to the product's attribute levels.

In [18]:
def utility(product, part_worths):
    pw = part_worths.to_numpy()
    ans = pw[1]
    for i in range(5):
        if product[i]:
            ans += pw[i+2]
    return ans

**Task 10:** Write a function `choose_product` that takes two arguments:
- `products`: a list of products, where each product is its own list as specified in Task 8, (Yes, it's a list of lists!)
- `part_worths`: a row of inidivudual part worths, just as used in Task 8.

This fuction should return a numpy array with the same length as the number of products; this array should contain a boolean value (true or false) for each product.  The value for a product should be true if it maximizes the utility for the individual; as multiple products can have equivalent utilities, more than one product might maximize utility.

As an example, if there are two products, the `choose_product` function could return `[True, False]` if the first product maximizes the utility, `[False, True]` if the second product maximizes utility, or `[True, True]` if both products have the same utility and therefor both maximize utility for specified the individual.

In [19]:
def choose_product(products, part_worths):
    score = []
    ans = []
    for product in products:
        score.append(utility(product, part_worths))
    maximum = max(score)
    for product in products:
        if (utility(product, part_worths) == maximum):
            ans.append(True)
        else:
            ans.append(False)
    return np.array(ans)

**Task 11:** Write a function `simulate` that takes two arguments:
- `products`: a list of products, identitcal to the argument for Task 9.
- The `individual_part_worths` data frame created for Task 7.

This function should simulate the market share of the products by having each individual choose a product from the list.  For an individual who chooses more than one item (multiple items are tied for the top utility value), their "vote" is split between the tied items.  The function should return an array of proportions of market share.  

In [20]:
def simulate(products, individual_part_worths):
    sim = np.empty((0,len(products)))
    for index, pw in individual_part_worths.iterrows():
        sim = np.append(sim, np.array([choose_product(products, pw).tolist()]), axis=0)
    sim = sim*1
    r,c = sim.shape
    for i in range(r):
        s = np.sum(sim[i])
        if s:
            sim[i] /= s
    ans = []
    for j in range(c):
        ans.append(np.sum(sim[:,j]) / np.sum(sim))
    return np.array(ans)

**Task 12:** Create a variable called `my_product` that has the ideal value (maximum average utility) based on what you discovered in Task 5; this should be in the format described in Task 9.  For example, if the two attributes were color (with levels red and green) and shape (with levels round and square), then `my_product` could take the value `["red", "square"]`.  Also create three competitiors (e.g., `competitor1`) using this same format.  Create a list called `all_products` that contains all four products and simulate the market with the products.

In [21]:
my_product = [True,True,True,False,True]
competitor1 = [True,False,True,False,True]
competitor2 = [True,False,True,False,False]
competitor3 = [True,True,False,False,True]
all_products = [my_product,competitor1,competitor2,competitor3]
simulate(all_products, individual_part_worths)

array([0.15151515, 0.31060606, 0.10606061, 0.43181818])

**Task 13:** Redefine `my_product` such that the most important attribute (from Task 6) now has the least liked level (per what you found in Task 5) and rerun the market simulation keeping the competitors the same.

In [22]:
my_product = [True,True,True,True,True]
competitor1 = [True,False,True,False,True]
competitor2 = [True,False,True,False,False]
competitor3 = [True,True,False,False,True] #based on top utility
all_products = [my_product,competitor1,competitor2,competitor3]

simulate(all_products, individual_part_worths)

array([0.09090909, 0.36363636, 0.11363636, 0.43181818])